# Purpose
This notebook describes the typical activities carried out  at the beginning to a project / thread when customer shares new data. We will be trying to understand the tables, columns and information flow. Typically we also look for data issues and confirm with respective owners for resolution. At the end of this activity, the data sources and their treatment is finalized. Code in this notebook will not be part of the production code.

This data can be downloaded from
[here](https://drive.google.com/file/d/11DqcBxxEcn3QA4YvPQmmExBm-m6AgUQ_/view?usp=sharing)

**NOTE**:
Download the data from the above link, and copy the extracted csv files to the path `data/raw/sales/` (relative to root of the code archive folder). Make sure to copy the files before continuing on with the rest of the notebook.

# Initialization

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
%%time

# Third-party imports
import os.path as op
import pandas as pd
import great_expectations as ge

# Project imports
from ta_lib.core.api import display_as_tabs, initialize_environment

# Initialization
initialize_environment(debug=False, hide_warnings=True)

CPU times: user 53 µs, sys: 12 µs, total: 65 µs
Wall time: 66.5 µs


# Data

## Background

Customer is a distributor of electronic devices. They partner with manufacturers, carriers and refurbishers and sell across to  retailers. The selling price is the outcome of negotiation between sales representatives and retailers. Customer wants to understand the selling price variation and determine  optimal pricing with Machine Learning.

In [34]:
from ta_lib.core.api import create_context, list_datasets, load_dataset

In [42]:
config_path = op.join('conf', 'config.yml')
context = create_context(config_path)

In [43]:
list_datasets(context)

['/raw/housing',
 '/cleaned/housing',
 '/train/housing/features',
 '/train/housing/target',
 '/test/housing/features',
 '/test/housing/target',
 '/score/housing/output']

In [45]:
# load datasets
housing_df = load_dataset(context, 'raw/housing')
# prod_df = load_dataset(context, 'raw/product')

# Exploratory Analysis

Given the raw data from data ingestion, we would now like to explore and learn more details about the data.


The output of the step would be a summary report and discussion of any pertinent findings.


In [46]:
# Import the eda API
import ta_lib.eda.api as eda

## Variable summary

In [47]:
display_as_tabs([('housing', housing_df.shape)])

Tabs
    [0] Str(tuple, name='housing')

In [49]:
sum1 = eda.get_variable_summary(housing_df)
# sum2 = eda.get_variable_summary(prod_df)

display_as_tabs([('housing', sum1)])

Tabs
    [0] DataFrame(height=300, name='housing', sizing_mode='fixed', value=                         V..., width=800)

In [50]:
housing_df.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

**Dev NOTES**

<details>
1. Datatypes : We have both numeric and other types. The bulk of them seem to be numeric. `Numeric` is defined to be one of [float|int|date] and the rest are categorized as `Others`. A column is assumed to have `date` values if it has the string `date` in the column name.


## Merging

We can merge orders table with prod table on SKU. Let us check first-cut cardinality issues. 

### Expected data validation rules

1. Quantity should be an integer
2. Quantity * UnitCost = SellingCost
3. Quantity * UnitPrice = SellingPrice

In [11]:
verification_dict = {}
orders_df = ge.from_pandas(orders_df)

#### Rule 1 verification

In [12]:
verification_dict["rule_1_check"] = orders_df.expect_column_values_to_be_of_type("Quantity", "int64", mostly=None, 
                                             result_format="BASIC", include_config=True).to_json_dict()

if verification_dict["rule_1_check"]["success"]:
    print("Rule 1 passed")
else:
    print("Rule 1 failed")

Rule 1 passed


#### Rule 2 verification

In [13]:
orders_df["selling_cal"] = orders_df["Quantity"] * orders_df["UnitCost"]
orders_df.selling_cal = orders_df.selling_cal.round()
orders_df["act_selling_round"] = orders_df.SellingCost.round()
verification_dict["rule_2_check"] = orders_df.expect_column_pair_values_to_be_equal("selling_cal", "act_selling_round", mostly=None, 
                                             result_format="BASIC", include_config=True).to_json_dict()

if verification_dict["rule_2_check"]["success"]:
    print("Rule 2 passed")
else:
    print("Rule 2 failed")

Rule 2 failed


#### Rule 3 verification

In [14]:
orders_df["selling_cal"] = orders_df["Quantity"] * orders_df["UnitPrice"]
orders_df.selling_cal = orders_df.selling_cal.round()
orders_df["act_selling_round"] = orders_df.SellingPrice.round()
verification_dict["rule_3_check"] = orders_df.expect_column_pair_values_to_be_equal("selling_cal", "act_selling_round", mostly=None, 
                                             result_format="BASIC", include_config=True).to_json_dict()

if verification_dict["rule_3_check"]["success"]:
    print("Rule 3 passed")
else:
    print("Rule 3 failed")

Rule 3 failed


#### Rule 2,3

Ideally by logic cost * units should give the total cost, however there are some orders where this condition is not matching. We should confirm these condition from client.

Similar goes for Price * units

In [15]:
# Back to Pandas
orders_df = orders_df.drop('selling_cal', axis=1)
orders_df = orders_df.drop('act_selling_round', axis=1)
orders_df = pd.DataFrame(orders_df)

### Table cardinality

In [16]:
from  ta_lib.core.api import setanalyse

setanalyse(orders_df.SKU.tolist(),prod_df.SKU.tolist())

{'A-B': 5, 'B-A': 0, 'AuB': 13549, 'A^B': 13544}

This represents a venn diagram on two lists. Left list is `A` & right one is `B`. `A-B` implies that are five SKUs in orders_df missing in product master. We can find them using unsimplied version.

Let us look at the set `A-B`.

In [17]:
missings_master_skus = setanalyse(orders_df.SKU.tolist(),prod_df.SKU.tolist(),simplify=False)['A-B']
missings_master_skus

{'APL AWS3AL(G) 42MM GLD CST',
 'APL AWS3AL(G+C) 38MM GRY CST',
 'SP3 SMS N920 NOTE5 32G GLD DST',
 'SP3 SMS S8+ G955 BLK BER SPT',
 'UNLKD SONY XPERIA XZS BLK 32GB'}

In [18]:
import numpy as np
print(('Records affected due to missing keys are {0} accounting to {1}% of orders').format(
    orders_df.SKU.isin(missings_master_skus).sum(),np.round(orders_df.SKU.isin(missings_master_skus).mean()*100,2)))

Records affected due to missing keys are 402 accounting to 0.23% of orders


**Since missing keys are very less we can proceed with inner join**

**Dev NOTES**

<details>
1. Cardinality with mulitple keys: If you have more than one key use setanalyse_df. You can pass dataframes instead of lists and the key columns
2. Excess master data (if `B-A` >0 in above example) will not be concern
3. If the `A-B` is larger, please check with client for alternative data sources. In case of left join keep a stragey for imputing.


### Master uniqueness

Product master is expected to have non duplicate primary keys. Let us verify them.

In [19]:
# Snippet 
df_freq = prod_df.SKU.value_counts().reset_index()
df_freq.columns = ['SKU','Frequency']
fil_ = df_freq.Frequency>1
if fil_.sum() > 0:
    print(("Found {0} duplicates in master. Sample duplicates are:").format(fil_.sum()))
    print(df_freq[fil_].head())
else:
    print("No duplciates in primary key")

Found 1 duplicates in master. Sample duplicates are:
                              SKU  Frequency
0  UNLKD SONY XPERIA XZS BLUE 32G          2


In [20]:
len(prod_df)

13545

In [21]:
# Dropping inconsistent records
print(("No. of rows before dropping duplicate SKUs: {0}".format(prod_df.shape[0])))
fil_ = (prod_df.SKU == 'UNLKD SONY XPERIA XZS BLUE 32G') & (prod_df.color.str.strip() == 'BLACK')
prod_df = prod_df[~fil_]
print(("No. of rows after dropping duplicate SKUs: {0}".format(prod_df.shape[0])))

No. of rows before dropping duplicate SKUs: 13545
No. of rows after dropping duplicate SKUs: 13544


## Health Analysis

Get an overview of the overall health of the dataset. This is usually quick to compute and hopefully highlights some problems to focus on.



### Summary Plot

Provides a high level summary of the dataset health.

**Watch out for:**

* too few numeric values
* high % of missing values
* high % of duplicate values
* high % of duplicate columns 

In [51]:
sum1, plot1 = eda.get_data_health_summary(housing_df, return_plot=True)
# sum2, plot2 = eda.get_data_health_summary(prod_df, return_plot=True)

display_as_tabs([('housing', plot1)])

Tabs
    [0] HoloViews(Layout, name='housing')

**Dev NOTES**

<details>
1. Datatypes : We have both numeric and other types. The bulk of them seem to be numeric. `Numeric` is defined to be one of [float|int|date] and the rest are categorized as `Others`. A column is assumed to have `date` values if it has the string `date` in the column name.

2. The missing value plot seems to indicate missing values are not present but we do have them. 

3. We are looking for duplicate observations (rows in the data). The plot shows the % of rows that are an exact replica of another row (using `df.duplicated`)

4. We are looking for duplicate features (columns in the data).

</details>

### Missing Values summary

This provides an overall view focussing on amount of missing values in the dataset.

**Watch out for:**
* A few columns have significant number of missing values 
* Most columns have significant number of missing values


In [23]:
sum1, plot1 = eda.get_missing_values_summary(orders_df, return_plot=True)
sum2, plot2 = eda.get_missing_values_summary(prod_df, return_plot=True)

display_as_tabs([('orders', plot1), ('product', plot2)])

tzname GB identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.



Tabs
    [0] HoloViews(Bars, name='orders')
    [1] HoloViews(Bars, name='product')

**Dev notes:**

<details>
    
    * By default, the following are considered missing/NA values : `[np.Nan, pd.NaT, 'NA', None]`
    * additional values can be passed to tigerml (add_additional_na_values)
    * these are applied to all columns.
    
    * some of the above information can be learnt from the data discovery step (see discussion below)
    
</details>

In [24]:
sum1 = eda.get_duplicate_columns(orders_df)
sum2 = eda.get_duplicate_columns(prod_df)

display_as_tabs([('orders', sum1), ('product', sum2)])

tzname GB identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.



Tabs
    [0] Markdown(str, name='orders')
    [1] Markdown(str, name='product')

In [25]:
sum1 = eda.get_outliers(orders_df)
sum2 = eda.get_outliers(prod_df)

display_as_tabs([('orders', sum1), ('product', sum2)])

tzname GB identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.



Tabs
    [0] DataFrame(height=300, name='orders', sizing_mode='fixed', value=              ('Data Shape..., width=800)
    [1] Markdown(str, name='product')

## Health Analysis report

Generate a report that has all the above data in a single html. This could be useful to submit to a client

In [53]:
from ta_lib.reports.api import summary_report

summary_report(housing_df, './housing.html')


prod: https://drive.google.com/file/d/1TM-T5HzAYpT8_1ugM5L6Bnfxp8r3uMem/view?usp=sharing

orders: https://drive.google.com/file/d/1uvehi90v1HFtScZrtWg3pW2zpi-DFQ1Y/view?usp=sharing
    